<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: jianzhang.Wu  
@Date: 2020-10-31 
@LastEditors: jianzhang.Wu  
@LastEditTime: 2020-10-31

In [1]:
import numpy as np
import cv2
import os,time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
import ipywidgets.widgets as widgets
from image_fun import bgr8_to_jpeg

In [2]:
#from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

#camera = USBCamera(width=320, height=240, capture_fps=30)
camera = CSICamera(width=320, height=240, capture_fps=30)

camera.running = True

In [3]:
image_widget = widgets.Image(format='jpg', width=320, height=240)
display(image_widget)
image_widget.value = bgr8_to_jpeg(camera.value)

Image(value=b'', format='jpg', height='240', width='320')

In [4]:
# Init tf model

MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09' #fast
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb' 
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90 
IMAGE_SIZE = (12, 8) 
fileAlreadyExists = os.path.isfile(PATH_TO_CKPT) 

if not fileAlreadyExists:
    print('Model does not exsist !')
    exit

In [5]:
# LOAD GRAPH
print('Loading...')
detection_graph = tf.Graph() 
with detection_graph.as_default(): #语句下定义属于计算图detection_graph的张量和操作
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
        serialized_graph = fid.read() 
        od_graph_def.ParseFromString(serialized_graph) 
        tf.import_graph_def(od_graph_def, name='')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True) 
category_index = label_map_util.create_category_index(categories)
print('Finish Load Graph..')

Loading...
Finish Load Graph..


In [6]:
print(type(category_index))

<class 'dict'>


In [7]:
print("dict['Name']: ", category_index[1]['name'])

dict['Name']:  person


In [ ]:
    # Main
    t_start = time.time()
    fps = 0

    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            while True:
                frame = camera.value
               # ret, frame = cap.read()
    #            frame = cv2.flip(frame, -1) # Flip camera vertically
    #             frame = cv2.resize(frame,(320,240))
                ##############
                image_np_expanded = np.expand_dims(frame, axis=0) 
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 
                detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 
                detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
                detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    #             print('Running detection..') 
                (boxes, scores, classes, num) = sess.run( 
                    [detection_boxes, detection_scores, detection_classes, num_detections], 
                    feed_dict={image_tensor: image_np_expanded}) 
    #             print('Done.  Visualizing..')
                vis_utils.visualize_boxes_and_labels_on_image_array(
                        frame,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        line_thickness=8)
        
                for i in range(0, 10):
                    if scores[0][i] >= 0.5:
                        print(category_index[int(classes[0][i])]['name'])
                ##############
                fps = fps + 1
                mfps = fps / (time.time() - t_start)
                cv2.putText(frame, "FPS " + str(int(mfps)), (10,10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
                image_widget.value = bgr8_to_jpeg(frame)


person
person
sports ball
sports ball
person
sports ball
person
sports ball
person
person
sports ball
person
sports ball
sports ball
person
person
sports ball
person
sports ball
sports ball
sports ball
frisbee
sports ball
frisbee
frisbee
frisbee
frisbee
frisbee
frisbee
frisbee
frisbee
frisbee
frisbee
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
sports ball
person
person
person
person
person
person
person
person
cat
person
person
person
person
person
person
person
person
person
person
person
remote
person
person
person
person
person
person
person
person
person
person
apple
person
person
person
person
person

2.通过创建摄像头的更新函数，当摄像头头值改变的时候进入更新函数对摄像头拍到的图片进行处理，以下为摄像头更新方式的创建，如果摄像头已经释放需要重新通过jetcam加载摄像头

In [9]:
detection_graph.as_default()
sess = tf.compat.v1.Session(graph=detection_graph)
    
t_start = time.time()
fps = 0

def update_image(change):
    global fps
    global sess
    frame = change['new']
    image_np_expanded = np.expand_dims(frame, axis=0)
    image_np_expanded = np.expand_dims(frame, axis=0) 
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
          
#    print('Running detection..') 
    (boxes, scores, classes, num) = sess.run( 
        [detection_boxes, detection_scores, detection_classes, num_detections], 
        feed_dict={image_tensor: image_np_expanded}) 
     
#    print('Done.  Visualizing..') 
    vis_utils.visualize_boxes_and_labels_on_image_array(
        frame,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
    for i in range(0, 10):
        if scores[0][i] >= 0.5:
            print(category_index[int(classes[0][i])]['name'])
    ##############
    fps = fps + 1
    mfps = fps / (time.time() - t_start)
    cv2.putText(frame, "FPS " + str(int(mfps)), (10,10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    image_widget.value = bgr8_to_jpeg(frame) 
    
    


以下为启动摄像头更新函数

In [10]:
camera.observe(update_image, names='value')

下方停止摄像头更新函数

In [ ]:
camera.unobserve(update_image, names='value')